# KFServing Pipeline samples

This notebook uses an older version of the KFServing Pipelines component meant for clusters using a KFServing version less than v0.5.0 which only supports the v1alpha2 API.

### Install the necessary kfp library

In [ ]:
!pip3 install kfp --upgrade

In [ ]:
import kfp.dsl as dsl
import kfp
from kfp import components
import json

# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client('kfserving_endpoint')
EXPERIMENT_NAME = 'KFServing Experiments'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

### TensorFlow example

Note: Change the action from `update` to `create` if you are deploying the model for the first time.

In [ ]:
kfserving_op = components.load_component_from_url(
  'https://raw.githubusercontent.com/kubeflow/pipelines/65bed9b6d1d676ef2d541a970d3edc0aee12400d/components/kubeflow/kfserving/component.yaml'
)

@dsl.pipeline(
  name='kfserving pipeline',
  description='A pipeline for kfserving.'
)
def kfservingPipeline(
    action = 'update',
    model_name='tf-sample',
    default_model_uri='gs://kfserving-samples/models/tensorflow/flowers',
    canary_model_uri='gs://kfserving-samples/models/tensorflow/flowers-2',
    canary_model_traffic_percentage='10',
    namespace='your_namespace',
    framework='tensorflow',
    default_custom_model_spec='{}',
    canary_custom_model_spec='{}',
    autoscaling_target='0',
    kfserving_endpoint=''
):

    # define workflow
    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             default_model_uri=default_model_uri,
                             canary_model_uri=canary_model_uri,
                             canary_model_traffic_percentage=canary_model_traffic_percentage,
                             namespace=namespace,
                             framework=framework,
                             default_custom_model_spec=default_custom_model_spec,
                             canary_custom_model_spec=canary_custom_model_spec,
                             autoscaling_target=autoscaling_target,
                             kfserving_endpoint=kfserving_endpoint).set_image_pull_policy('Always')

# Compile pipeline
import kfp.compiler as compiler
compiler.Compiler().compile(kfservingPipeline, 'tf-flower.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'tf-flower', 'tf-flower.tar.gz')

### Custom model example

In [ ]:
kfserving_op = components.load_component_from_url(
  'https://raw.githubusercontent.com/kubeflow/pipelines/65bed9b6d1d676ef2d541a970d3edc0aee12400d/components/kubeflow/kfserving/component.yaml'
)

@dsl.pipeline(
  name='kfserving pipeline',
  description='A pipeline for kfserving.'
)
def kfservingPipeline(
    action = 'update',
    model_name='custom-sample',
    default_model_uri='',
    canary_model_uri='',
    canary_model_traffic_percentage='0',
    namespace='kubeflow',
    framework='custom',
    default_custom_model_spec='{"name": "image-segmenter", "image": "codait/max-image-segmenter:latest", "port": "5000"}',
    canary_custom_model_spec='{}',
    autoscaling_target='0',
    kfserving_endpoint=''
):

    # define workflow
    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             default_model_uri=default_model_uri,
                             canary_model_uri=canary_model_uri,
                             canary_model_traffic_percentage=canary_model_traffic_percentage,
                             namespace=namespace,
                             framework=framework,
                             default_custom_model_spec=default_custom_model_spec,
                             canary_custom_model_spec=canary_custom_model_spec,
                             autoscaling_target=autoscaling_target,
                             kfserving_endpoint=kfserving_endpoint).set_image_pull_policy('Always')

# Compile pipeline
import kfp.compiler as compiler
compiler.Compiler().compile(kfservingPipeline, 'custom.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'custom-model', 'custom.tar.gz')